<a href="https://colab.research.google.com/github/Norod/my-colab-experiments/blob/master/Multiple_input_files_demo_3ddfa_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A simple demostration of how to run

In [ ]:
# if the repo is not there, it will clone it
%cd /content
!git clone https://github.com/cleardusk/3DDFA_V2.git
%cd 3DDFA_V2

/content
Cloning into '3DDFA_V2'...
remote: Enumerating objects: 669, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 669 (delta 4), reused 10 (delta 2), pack-reused 652
Receiving objects: 100% (669/669), 74.77 MiB | 30.49 MiB/s, done.
Resolving deltas: 100% (345/345), done.
/content/3DDFA_V2


In [ ]:
!sh ./build.sh

running build_ext
cythoning nms/cpu_nms.pyx to nms/cpu_nms.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/3DDFA_V2/FaceBoxes/utils/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'nms.cpu_nms' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/nms
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c nms/cpu_nms.c -o build/temp.linux-x86_64-3.7/nms/cpu

In [ ]:
# before import, make sure FaceBoxes and Sim3DR are built successfully, e.g.,

import cv2
import yaml

from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.render import render
from utils.depth import depth
from utils.pncc import pncc
from utils.uv import uv_tex
from utils.pose import viz_pose
from utils.serialization import ser_to_ply, ser_to_obj
from utils.functions import draw_landmarks, get_suffix

import matplotlib.pyplot as plt
from skimage import io

### Load configs

In [ ]:
# load config
cfg = yaml.load(open('configs/mb1_120x120.yml'), Loader=yaml.SafeLoader)

# Init FaceBoxes and TDDFA, recommend using onnx flag
onnx_flag = True  # or True to use ONNX to speed up
if onnx_flag:
    !pip install onnxruntime
    
    import os
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
    os.environ['OMP_NUM_THREADS'] = '4'
    from FaceBoxes.FaceBoxes_ONNX import FaceBoxes_ONNX
    from TDDFA_ONNX import TDDFA_ONNX

    face_boxes = FaceBoxes_ONNX()
    tddfa = TDDFA_ONNX(**cfg)
else:
    face_boxes = FaceBoxes()
    tddfa = TDDFA(gpu_mode=False, **cfg)

     |████████████████████████████████| 4.1MB 8.8MB/s 


/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1193: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input
  'Automatically generated names will be applied to each dynamic axes of input {}'.format(key))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1193: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input output
  'Automatically generated names will be applied to each dynamic axes of input {}'.format(key))


Convert /content/3DDFA_V2/FaceBoxes/weights/FaceBoxesProd.pth to /content/3DDFA_V2/FaceBoxes/weights/FaceBoxesProd.onnx done.


/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1193: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input alpha_shp
  'Automatically generated names will be applied to each dynamic axes of input {}'.format(key))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1193: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input alpha_exp
  'Automatically generated names will be applied to each dynamic axes of input {}'.format(key))


Convert configs/bfm_noneck_v3.pkl to configs/bfm_noneck_v3.onnx done.
weights/mb1_120x120.onnx does not exist, try to convert the `.pth` version to `.onnx` online
Convert weights/mb1_120x120.pth to weights/mb1_120x120.onnx done.


In [ ]:
FOLDER_WITH_IMAGES = '/content/3DDFA_V2/examples/custom_inputs/'
!cd /content/3DDFA_V2

!mkdir -p $FOLDER_WITH_IMAGES

print ("Please place your images in " + str(FOLDER_WITH_IMAGES))

Please place your images in /content/3DDFA_V2/examples/custom_inputs/


In [ ]:

import os
import cv2
 
files = [f for f in os.listdir(FOLDER_WITH_IMAGES) if f.endswith('.jpg')]

# -o can be: 2d_sparse 2d_dense 3d depth pncc pose uv_tex ply obj
for infile in files: 
   print('Processing file %s ...' % (infile))
   full_path = os.path.join(FOLDER_WITH_IMAGES, infile)   
   !python demo.py -f $full_path -o obj --show_flag=false --onnx;
 
print('Done')

Processing file input_img_00008.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00008_obj.obj
Processing file input_img_00009.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00009_obj.obj
Processing file input_img_00003.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00003_obj.obj
Processing file input_img_00005.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00005_obj.obj
Processing file input_img_00007.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00007_obj.obj
Processing file input_img_00001.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00001_obj.obj
Processing file input_img_00004.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00004_obj.obj
Processing file input_img_00006.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00006_obj.obj
Processing file input_img_00002.jpg ...
Detect 1 faces
Dump tp examples/results/input_img_00002_obj.obj
Done


In [ ]:
%cd /content/3DDFA_V2/examples/results
!zip /content/3ddfa_v2_results.zip ./*

/content/3DDFA_V2/examples/results
  adding: input_img_00001_obj.obj (deflated 74%)
  adding: input_img_00002_obj.obj (deflated 73%)
  adding: input_img_00003_obj.obj (deflated 73%)
  adding: input_img_00004_obj.obj (deflated 74%)
  adding: input_img_00005_obj.obj (deflated 73%)
  adding: input_img_00006_obj.obj (deflated 73%)
  adding: input_img_00007_obj.obj (deflated 74%)
  adding: input_img_00008_obj.obj (deflated 73%)
  adding: input_img_00009_obj.obj (deflated 74%)
  adding: videos/ (stored 0%)


In [ ]:
print("Please download the zip file from /content/3ddfa_v2_results.zip")

Please download the zip file from /content/3ddfa_v2_results.zip
